In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [5]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [6]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [7]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.densenet121(pretrained=False)
num_ftrs = model_conv.classifier.in_features
maxm = 0
for i, param in model_conv.named_parameters():
    maxm = maxm + 1
    if maxm < 182:
      param.requires_grad = False
print(maxm)
#model_conv.fc = nn.Sequential(
#            nn.Linear(512,400),
#            nn.Dropout(0.4),
#            nn.Conv2d(400,num_ftrs,8),
#            nn.MaxPool2d(3, 3),
#            nn.Linear(num_ftrs, 6))
model_conv.fc = nn.Linear(num_ftrs,6)
model_conv = model_conv.cuda()

c:\users\rohit\virtualenv\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


364


In [9]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)




In [12]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [15]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=2)

Epoch 0/1
----------



 15%|███████████▉                                                                     | 67/453 [00:18<01:44,  3.68it/s]


 30%|███████████████████████▊                                                        | 135/453 [00:37<01:24,  3.74it/s]


 45%|███████████████████████████████████▊                                            | 203/453 [00:56<01:09,  3.58it/s]


 60%|███████████████████████████████████████████████▊                                | 271/453 [01:16<00:58,  3.11it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [02:05<00:00,  3.90it/s]

phase Loss: 0.0127 Acc: 0.8522



 84%|████████████████████████████████████████████████████████████████████▋             | 67/80 [00:10<00:02,  5.93it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:12<00:00,  6.43it/s]

phase Loss: 0.0166 Acc: 0.8180
0.0005
Training complete in 4m 38s
Best val Acc: 0.828180


In [26]:
def test_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs, labels_ = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        labels.append(labels_)

    return predictions, labels
  
predictions, labels = test_model(model_conv, val_loader, optimizer_conv, True)




  0%|                                                                                           | 0/80 [00:00<?, ?it/s]


  1%|█                                                                                  | 1/80 [00:00<00:12,  6.15it/s]


  2%|██                                                                                 | 2/80 [00:00<00:12,  6.37it/s]


  4%|███                                                                                | 3/80 [00:00<00:12,  6.32it/s]


  5%|████▏                                                                              | 4/80 [00:00<00:11,  6.45it/s]


  6%|█████▏                                                                             | 5/80 [00:00<00:11,  6.57it/s]


  8%|██████▏                                                                            | 6/80 [00:00<00:10,  6.74it/s]


  9%|███████▎                                                                           | 7/80 [00:01<00:10,  6.67it/s]


 10%|████████▎       

In [41]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'densenet121_sequential.pth')

In [39]:
predictions1, labels1 = torch.cat(predictions), torch.cat(labels)
predictions1, labels1 = np.array(predictions1.cpu()), np.array(labels1)

In [40]:
predictions1

array([3, 4, 0, ..., 5, 5, 4], dtype=int64)

In [41]:
labels1

array([3, 4, 0, ..., 5, 5, 4], dtype=int64)

In [42]:
len(labels1)

2555

In [31]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [32]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:32<00:00,  7.45it/s]


In [33]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [34]:
len(predictions3)

7301

In [35]:
predictions3[:5]

array([5, 0, 4, 2, 5], dtype=int64)

In [36]:
result = pd.read_csv('resnet50_15_col.csv')

In [37]:
result.shape[0]

7301

In [38]:
result['label'] = predictions3

In [39]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,2
4,14.jpg,5


In [40]:
result.to_csv('torch_densenet121_sequential.csv')

# ResNet101

In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [3]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [4]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [5]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.resnet101(pretrained='imagenet')
#num_ftrs = model_conv.classifier.in_features
#maxm = 0
#for i, param in model_conv.named_parameters():
#    maxm = maxm + 1
#    if maxm < 182:
#      param.requires_grad = False
#print(maxm)
#model_conv.classifier = nn.Linear(num_ftrs, 6)
#


In [7]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
    if maxm < 135:
        param.requires_grad = False
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 6)
model_conv = model_conv.cuda()

In [8]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)


In [11]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs), Variable(labels)
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [13]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=25)

Epoch 0/24
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [03:21<00:00,  2.58it/s]


phase Loss: 0.0058 Acc: 0.9333


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:13<00:00,  5.90it/s]


phase Loss: 0.0080 Acc: 0.9205
0.0005
Epoch 1/24
----------


...

phase Loss: 0.0101 Acc: 0.9378
7.8125e-06
Epoch 24/24
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [03:21<00:00,  2.63it/s]


phase Loss: 0.0002 Acc: 0.9982


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:13<00:00,  5.88it/s]


phase Loss: 0.0108 Acc: 0.9386
7.8125e-06
Training complete in 90m 3s
Best val Acc: 0.940900


In [14]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'resnet101.pth')

In [15]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [16]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:35<00:00,  4.73it/s]


In [17]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [18]:
len(predictions3)

7301

In [19]:
predictions3[:5]

array([5, 0, 4, 4, 5], dtype=int64)

In [20]:
result = pd.read_csv('resnet50_15_col.csv')

In [21]:
result.shape[0]

7301

In [22]:
result['label'] = predictions3

In [23]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,4
4,14.jpg,5


In [24]:
result.to_csv('torch_resnet101.csv')

# ResNet152

In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [3]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [4]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [5]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.resnet152(pretrained='imagenet')
#num_ftrs = model_conv.classifier.in_features
#maxm = 0
#for i, param in model_conv.named_parameters():
#    maxm = maxm + 1
#    if maxm < 182:
#      param.requires_grad = False
#print(maxm)
#model_conv.classifier = nn.Linear(num_ftrs, 6)
#


In [11]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
print(maxm)

467


In [12]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
    if maxm < 445:
        param.requires_grad = False
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 6)
model_conv = model_conv.cuda()

In [13]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)


In [16]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [18]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=5)

Epoch 0/4
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:54<00:00,  3.94it/s]


phase Loss: 0.0053 Acc: 0.9391


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:17<00:00,  4.59it/s]


phase Loss: 0.0078 Acc: 0.9162
0.0005
Epoch 1/4
----------


...

phase Loss: 0.0071 Acc: 0.9284
0.0005
Epoch 4/4
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:56<00:00,  3.91it/s]


phase Loss: 0.0040 Acc: 0.9543


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:17<00:00,  4.58it/s]


phase Loss: 0.0068 Acc: 0.9319
0.0005
Training complete in 11m 5s
Best val Acc: 0.931898


In [19]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'resnet152.pth')

In [20]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [21]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:48<00:00,  4.56it/s]


In [22]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [23]:
len(predictions3)

7301

In [24]:
predictions3[:5]

array([5, 0, 4, 2, 5], dtype=int64)

In [25]:
result = pd.read_csv('resnet50_15_col.csv')

In [26]:
result.shape[0]

7301

In [27]:
result['label'] = predictions3

In [23]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,4
4,14.jpg,5


In [28]:
result.to_csv('torch_resnet152.csv')

# Densenet169

In [2]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [4]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [5]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [6]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.densenet169(pretrained='imagenet')
#num_ftrs = model_conv.classifier.in_features
#maxm = 0
#for i, param in model_conv.named_parameters():
#    maxm = maxm + 1
#    if maxm < 182:
#      param.requires_grad = False
#print(maxm)
#model_conv.classifier = nn.Linear(num_ftrs, 6)
#


c:\users\rohit\virtualenv\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to C:\Users\rohit/.torch\models\densenet169-b2777c0a.pth
57365526it [00:48, 1183761.48it/s]


In [8]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
print(maxm)

508


In [32]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
    if maxm < 470:
        param.requires_grad = False
num_ftrs = model_conv.classifier.in_features
model_conv.classifier = nn.Linear(num_ftrs, 6)
model_conv = model_conv.cuda()

In [36]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)


In [37]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs), Variable(labels)
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [35]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=3)

Epoch 0/2
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:29<00:00,  5.41it/s]


phase Loss: 0.0527 Acc: 0.3683


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:12<00:00,  5.97it/s]


phase Loss: 0.0484 Acc: 0.5499
5e-06
Epoch 1/2
----------


...

phase Loss: 0.0418 Acc: 0.7695
5e-06
Epoch 2/2
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:30<00:00,  5.55it/s]


phase Loss: 0.0395 Acc: 0.8158


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:13<00:00,  6.42it/s]


phase Loss: 0.0358 Acc: 0.8575
5e-06
Training complete in 5m 11s
Best val Acc: 0.857534


In [17]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'densenet169.pth')

In [18]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [19]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:33<00:00,  7.22it/s]


In [20]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [21]:
len(predictions3)

7301

In [22]:
predictions3[:5]

array([5, 0, 4, 4, 5], dtype=int64)

In [23]:
result = pd.read_csv('resnet50_15_col.csv')

In [24]:
result.shape[0]

7301

In [25]:
result['label'] = predictions3

In [26]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,4
4,14.jpg,5


In [27]:
result.to_csv('torch_densenet169.csv')

# SqueezeNet

In [38]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [39]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [40]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [41]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [42]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.squeezenet1_1(True)
#num_ftrs = model_conv.classifier.in_features
#maxm = 0
#for i, param in model_conv.named_parameters():
#    maxm = maxm + 1
#    if maxm < 182:
#      param.requires_grad = False
#print(maxm)
#model_conv.classifier = nn.Linear(num_ftrs, 6)
#


Downloading: "https://download.pytorch.org/models/squeezenet1_1-f364aa15.pth" to C:\Users\rohit/.torch\models\squeezenet1_1-f364aa15.pth
4966400it [00:02, 2388291.88it/s]


In [50]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
print(maxm)

52


In [52]:
maxm = 0
for param in model_conv.parameters():
    maxm = maxm+1
    if maxm < 30:
        param.requires_grad = False
num_ftrs = model_conv.classifier.in_features
model_conv.classifier = nn.Linear(num_ftrs, 6)
model_conv = model_conv.cuda()

AttributeError: 'Sequential' object has no attribute 'in_features'

In [13]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)


In [16]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [18]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=5)

Epoch 0/4
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:54<00:00,  3.94it/s]


phase Loss: 0.0053 Acc: 0.9391


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:17<00:00,  4.59it/s]


phase Loss: 0.0078 Acc: 0.9162
0.0005
Epoch 1/4
----------


...

phase Loss: 0.0071 Acc: 0.9284
0.0005
Epoch 4/4
----------


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [01:56<00:00,  3.91it/s]


phase Loss: 0.0040 Acc: 0.9543


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:17<00:00,  4.58it/s]


phase Loss: 0.0068 Acc: 0.9319
0.0005
Training complete in 11m 5s
Best val Acc: 0.931898


In [19]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'resnet152.pth')

In [20]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [21]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:48<00:00,  4.56it/s]


In [22]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [23]:
len(predictions3)

7301

In [24]:
predictions3[:5]

array([5, 0, 4, 2, 5], dtype=int64)

In [25]:
result = pd.read_csv('resnet50_15_col.csv')

In [26]:
result.shape[0]

7301

In [27]:
result['label'] = predictions3

In [23]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,4
4,14.jpg,5


In [28]:
result.to_csv('torch_resnet152.csv')

# DenseNet on Places365

In [16]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [28]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image

arch = 'resnet50'
model_weights = '%s_places365.pth.tar' % arch

model_conv = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_weights, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model_conv.load_state_dict(state_dict)
model_conv.eval()

#model_conv = torchvision.models.resnet50()
#model_conv.load_state_dict(torch.load(model_weight)['state_dict'])


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [31]:

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        #transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.Scale(256),
        transforms.Resize((224, 224)),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
transformed_dataset_train =  SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, 
                                    transform=data_transforms['train']
                                           )      
train_dataloader = DataLoader(transformed_dataset_train, batch_size=32,
                        shuffle=False, num_workers=0)
validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)
   
batch_size = 32
dataloader = {'train': train_dataloader,
             'val': val_loader
             }
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def train_model (model, criterion, optimizer, scheduler, num_epochs, total_steps):
    since = time.time()

    print('total_steps is %d' % total_steps)
    mystep = 0

    best_model_wts = model.state_dict()
    best_acc = 0.0


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        if (epoch%10 == 0):
            torch.save(best_model_wts, ('%s_model_wts_%d.pth')% (arch, epoch))

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            top1 = AverageMeter()
            top3 = AverageMeter()

            # Iterate over data.
            for data in dataloader[phase]:
                # get the inputs
                mystep = mystep + 1
                if(mystep%100 ==0):
                    duration = time.time() - since
                    print('step %d vs %d in %.0f s' % (mystep, total_steps, duration))

                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)
#                print(type(labels)) # <class 'torch.autograd.variable.Variable'>
#                print(type(labels.data)) # <class 'torch.cuda.LongTensor'>
                prec1, prec3 = accuracy(outputs.data, labels.data, topk=(1, 3))
                top1.update(prec1[0], inputs.data.size(0))
                top3.update(prec3[0], inputs.data.size(0))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            #没测试batch_size不能被dataset_size整除时会不会有问题
            print('{} Loss: {:.6f} Acc: {:.6f}'.format(
                phase, epoch_loss, epoch_acc))
            print(' * Prec@1 {top1.avg:.6f} Prec@3 {top3.avg:.6f}'.format(top1=top1, top3=top3))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

        #if (epoch%10 == 0):
           # torch.save(best_model_wts, ('models/best_model_wts_%d.pth')% epoch)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.6f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

'''trained layers'''
#model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 6)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss().cuda()

'''optimizer'''
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.0001, momentum=0.9)

'''Decay LR by a factor of 0.1 every 100 epochs'''
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=100, gamma=0.1)


######################################################################
# Train and evaluate

num_epochs = 2
total_steps = 1.0 * num_epochs * (dataset_sizes['train'] + dataset_sizes['val']) / batch_size
print(total_steps)
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs, total_steps)
torch.save(model_conv.state_dict(), ('%s_best_model_wts_final.pth')%arch)


1064.625
total_steps is 1064
Epoch 0/1
----------
step 100 vs 1064 in 12 s
step 200 vs 1064 in 23 s
step 300 vs 1064 in 36 s
step 400 vs 1064 in 47 s
step 500 vs 1064 in 59 s
train Loss: 0.044784 Acc: 0.000000
 * Prec@1 74.944229 Prec@3 94.405312
step 600 vs 1064 in 70 s
val Loss: 0.044102 Acc: 0.000000
 * Prec@1 46.731899 Prec@3 80.939339

Epoch 1/1
----------
step 700 vs 1064 in 82 s
step 800 vs 1064 in 93 s
step 900 vs 1064 in 106 s
step 1000 vs 1064 in 117 s
step 1100 vs 1064 in 129 s
train Loss: 0.026804 Acc: 1.000000
 * Prec@1 87.166847 Prec@3 99.195732
step 1200 vs 1064 in 140 s
val Loss: 0.040639 Acc: 0.000000
 * Prec@1 52.367908 Prec@3 82.583176

Training complete in 2m 23s
Best val Acc: 0.000000


In [9]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)




In [12]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [15]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=2)

Epoch 0/1
----------



 30%|███████████████████████▊                                                        | 135/453 [00:37<01:24,  3.74it/s]


 60%|███████████████████████████████████████████████▊                                | 271/453 [01:16<00:58,  3.11it/s]


 90%|███████████████████████████████████████████████████████████████████████▉        | 407/453 [01:54<00:12,  3.61it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [02:07<00:00,  3.96it/s]

...

 90%|███████████████████████████████████████████████████████████████████████▉        | 407/453 [01:52<00:12,  3.69it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [02:05<00:00,  3.90it/s]

phase Loss: 0.0127 Acc: 0.8522



100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:12<00:00,  6.43it/s]

phase Loss: 0.0166 Acc: 0.8180
0.0005
Training complete in 4m 38s
Best val Acc: 0.828180


In [32]:
def test_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs, labels_ = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        labels.append(labels_)

    return predictions, labels
  
predictions, labels = test_model(model_conv, val_loader, optimizer_conv, True)


 84%|████████████████████████████████████████████████████████████████████▋             | 67/80 [00:08<00:01,  8.51it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:09<00:00,  8.77it/s]

In [33]:
#model_path = 'C:\Users\rohit\VirtualEnv'
torch.save(model_conv.state_dict(), 'resnet50_places365.pth')

In [39]:
predictions1, labels1 = torch.cat(predictions), torch.cat(labels)
predictions1, labels1 = np.array(predictions1.cpu()), np.array(labels1)

In [40]:
predictions1

array([3, 4, 0, ..., 5, 5, 4], dtype=int64)

In [41]:
labels1

array([3, 4, 0, ..., 5, 5, 4], dtype=int64)

In [42]:
len(labels1)

2555

In [34]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [35]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)


 29%|███████████████████████▋                                                         | 67/229 [00:08<00:22,  7.25it/s]


 59%|███████████████████████████████████████████████▏                                | 135/229 [00:17<00:13,  7.06it/s]


 89%|██████████████████████████████████████████████████████████████████████▉         | 203/229 [00:25<00:03,  7.55it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:29<00:00,  7.80it/s]

In [36]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [39]:
len(predictions3)

7301

In [40]:
predictions3[:5]

array([5, 0, 4, 2, 5], dtype=int64)

In [41]:
result = pd.read_csv('submissresnet50_15_col.csv')

FileNotFoundError: [Errno 2] File b'resnet50_15_col.csv' does not exist: b'resnet50_15_col.csv'

In [37]:
result.shape[0]

7301

In [38]:
result['label'] = predictions3

In [39]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,2
4,14.jpg,5


In [40]:
result.to_csv('torch_resnet50_places365.csv',index=False)